In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import h5py
from gensim.models import Word2Vec
import gensim.models
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.models import Sequential
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense, Dropout, Activation, Bidirectional
from keras.layers import LSTM, SimpleRNN, GRU
import re
from nltk.tokenize import word_tokenize
#import preprocess
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
## import word2vec with google
WORD2VEC_VECTORS_BIN  =  "/content/gdrive/My Drive/SFResearch/Clickbait_Datasets/GoogleNews-vectors-negative300.bin.gz"

w2v  =  gensim . models . KeyedVectors . load_word2vec_format ( WORD2VEC_VECTORS_BIN , binary = True )
stop_words  =  set ( stopwords . words ( 'english' ))


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


OSError: ignored

In [ ]:
## Data cleaning
df [ 'cleaned_tweet' ] =  df [ 'text' ]. apply ( preprocess . clean_tweet )

## to find the maximum number of words in a sentence in order to determine the sequence size
L = []
for  i , token  in  enumerate ( df [ 'cleaned_tweet' ]):
    word = [ w  for  w  in  token . split () if  not  w  in  stop_words ]
    L . append ( len ( word ))
    
sequence_size = max ( L )

## separate data non-randomly to avoid having a problem during cross validation due to indexes

split = int ( 7 * len ( df ) / 8 )

text_train , text_test , y_train , y_test = df . iloc [: split , 2 ], df . ILOC [ split :, 2 ], df . iloc [: split , 1 ], df . ILOC [ split :, 1 ]

In [ ]:
## apply the word2 with google but WORD by WORD !! the problem is that not all the words will freeze in the word2vec

dimsize = 300
def  compute_matrix ( text ):
    X = np . zeros (( len ( text ), sequence_size , dimsize ))


    for  i , token  in  enumerate ( text ):
        word = token . split ()
        try :
        
            j = 0
            for  w  in  word :
                if  w  not  in  stop_words :
                    X [ i , j ] = w2v [ w ]
                    j + = 1
        except :
            pass
    return  X 

X_train = compute_matrix ( text_train )
X_test = compute_matrix ( text_test )

SyntaxError: ignored

In [ ]:
## create the model 
def  creat_model_RNN ():
    
    model  =  Sequential ()
    model . add ( Bidirectional ( SimpleRNN ( 128 , dropout = 0.1 ), input_shape = ( sequence_size , dimsize )))   # try using a GRU and a SimpleRNN
    model . add ( Dense ( 1 ))
    model . add ( Activation ( 'sigmoid' ))
    model . compile ( loss = 'binary_crossentropy' , optimizer = 'rmsprop' , metrics = [ 'accuracy' ])
    
    return  model

def  creat_model_GRU ():
    
    model  =  Sequential ()
    model . add ( Bidirectional ( GRU ( 128 , dropout = 0.1 ), input_shape = ( sequence_size , dimsize )))   # try using a GRU and a SimpleRNN
    model . add ( Dense ( 1 ))
    model . add ( Activation ( 'sigmoid' ))
    model . compile ( loss = 'binary_crossentropy' , optimizer = 'rmsprop' , metrics = [ 'accuracy' ])
    
    return  model

def  creat_model_LSTM ():
    
    model  =  Sequential ()
    model . add ( Bidirectional ( LSTM ( 128 , dropout = 0.1 ), input_shape = ( sequence_size , dimsize )))   # try using a GRU and a SimpleRNN
    model . add ( Dense ( 1 ))
    model . add ( Activation ( 'sigmoid' ))
    model . compile ( loss = 'binary_crossentropy' , optimizer = 'rmsprop' , metrics = [ 'accuracy' ])
    
    return  model

## define a file that saves the weights of the best model
cp = ModelCheckpoint ( "best_model.h5" , verbose = 1 , save_best_only = True )
earlystop_cb  =  keras . callbacks . EarlyStopping ( monitor = 'val_loss' , patience = 12 , verbose = 1 , mode = 'auto' )
 
callback_list = [ cp , earlystop_cb ]  

# try using different optimizers and different optimizer configs
def  fit_model ( model , X_train , y_train , x_valid , y_valid , batch_size  =  batch_size ):

    #earlystop_cb = keras.callbacks.EarlyStopping (monitor = 'val_loss', patience = 7, verbose = 1, mode = 'auto')

    model . fit ( X_train , y_train , batch_size = batch_size , epochs = 20 , validation_data = [ x_valid , y_valid ], callbacks = callback_list )
    #score, acc = model.evaluate (X_test, y_test, batch_size = batch_size)
    
    return  model

In [ ]:
## cross_validation 10folds
batch_size = 64
n_folds  =  10
skf  =  StratifiedKFold ( n_splits = n_folds , shuffle = True )


for  i , ( train , valid ) in  enumerate ( skf . split ( X_train , y_train )):
    print ( X_train [ train ]. shape , y_train [ train ]. shape , X_train [ valid ]. shape , y_train [ valid ]. shape )
    print ( "Running Fold" , i + 1 , "/" , n_folds )
    model  =  None  # Clearing the NN.
    model  =  creat_model_RNN ()
    hist = fit_model ( model , X_train [ train ], y_train [ train ], X_train [ valid ], y_train [ valid ])

In [ ]:
## you can plot the loss function for train and valid per epoch
loss_train = hist . history [ 'loss' ]
loss_val = hist . history [ 'val_loss' ]
plt . plot ( loss_train , "b" , label = "loss_train" )
plt . plot ( loss_val , "r" , label = "loss_valid" )
plt . title ( "loss over training epochs" )
plt . legend ()
plt . show ()    

In [ ]:
# save the weights of the best model to use them in the test
hist . load_weights ( "best_model.h5" )

##EVALUATION
score , acc  =  hist . evaluate ( X_test , y_test ,
                            batch_size = batch_size )
print ( 'Test score:' , score )
print ( 'Test accuracy:' , acc )
#prediction
y_pred  =  model . predict_classes ( X_test )
y_scores  =  model . predict_proba ( X_test )

## display prediction results
roc  =  roc_auc_score ( y_test , y_scores )
print ( 'ROC score:' , roc )

metrics  =  classification_report ( y_test , y_pred , digits = 4 )
print ( 'Classification Report \ n ' )
print ( metrics )

cm  =  confusion_matrix ( y_test , y_pred )
print ( 'Confusion Matrix \ n ' )
print ( cm )